In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16738548
paper_name = 'brown_brown_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/inline-supplementary-material-1.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4297 x 55


In [7]:
original_data.head()

,20040824,NAME,WEIGHT,DESC,AAPO,YPD,Wort,RafA,Lova,HygB,...,SC,Sorb-5g,Sorb-15g,Thr,Tpt,TPZ,Trp,UVA,UVB,UVC
0,YML095C,RAD10,1,removal of nonhomologous ends*,-0.480,0.014,-0.233,-0.137,0.035,0.063,...,-0.210,-0.085,-0.152,-0.211,-0.045,-1.540,-0.171,-0.248,-3.200,-3.082
1,YER162C,RAD4,1,"nucleotide-excision repair, DNA damage recogni...",-0.091,0.047,-0.059,0.052,0.054,0.106,...,-0.220,-0.093,-0.042,-0.201,0.128,-1.301,-0.239,-0.230,-3.296,-3.205
2,YLR032W,RAD5,1,DNA repair,-0.134,-0.544,-0.105,0.047,0.117,-0.257,...,-0.039,0.327,0.388,0.233,-0.046,-1.182,-0.074,-0.332,-2.529,-2.556
3,YMR201C,RAD14,1,"nucleotide-excision repair, DNA damage recogni...",-0.242,0.085,0.005,0.034,-0.078,-0.034,...,-0.234,-0.136,-0.224,-0.401,-0.208,-1.135,-0.220,0.178,-2.912,-2.762
4,YGR258C,RAD2,1,"nucleotide-excision repair, DNA incision, 3'-t...",-0.044,0.016,0.080,-0.060,0.131,0.047,...,-0.115,-0.066,0.007,-0.089,0.087,-0.675,-0.100,-0.098,-3.555,-3.417


In [8]:
original_data['orf'] = original_data['20040824'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [20040824, NAME, WEIGHT, DESC, AAPO, YPD, Wort, RafA, Lova, HygB, GlyE, Calc, Caff, AntA, ActD, Alk-5g, Alk-15g, AraC, Arn, Ars, Ben, Bleo, Cis1, Cis4, CPTa, CPTc, Dox, Gal-5g, Gal-15g, H2O2, HU, Ida, IR, Lys, Mech, Mel, Min-5g, Min-15g, MMC, MMS, NaCl-5g, NaCl-15g, Nys-5g, Nys-15g, Oxa, SC, Sorb-5g, Sorb-15g, Thr, Tpt, TPZ, Trp, UVA, UVB, UVC, orf]
Index: []

[0 rows x 56 columns]


In [12]:
original_data.set_index('orf', inplace=True)
original_data.drop(columns=['20040824','NAME','WEIGHT','DESC'], inplace=True)

In [13]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(4258, 51)

# Load dataset ids

In [18]:
dt = pd.read_csv('extras/dataset_id_name.txt', sep='\t', header=None)

In [19]:
dt.head()

,0,1
0,499,AAPO
1,4751,ActD
2,4752,AntA
3,4753,AraC
4,4755,Ars


In [20]:
dt.set_index(1, inplace=True)

In [21]:
dt = dt.reindex(index=original_data.columns.values)

In [24]:
dt = dt.loc[dt[0].notnull()]
dt[0] = dt[0].astype(int)

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
dt

,0
1,
AAPO,499
YPD,4776
Wort,4775
RafA,4771
Lova,4767
HygB,4765
GlyE,4777
Calc,4759
Caff,4758


In [26]:
dataset_ids = dt[0].values

In [29]:
original_data = original_data.reindex(columns=dt.index.values)

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,499,4776,4775,4771,4767,4765,4777,4759,4758,4752,...,4762,4763,4764,4766,4768,4769,4770,4772,4773,4774
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL002W,-1.347,-0.308,0.117,-0.425,-0.327,-1.672,-0.287,-1.268,-0.663,-0.001,...,-0.755,-0.247,-0.380,-0.331,-0.129,-0.118,0.005,-0.035,-0.920,-0.251
YAL004W,-0.167,0.052,-0.102,-0.144,-0.144,-0.055,-0.003,0.110,0.079,0.205,...,0.270,0.420,0.185,-0.058,-0.106,-0.028,0.108,-0.046,0.054,-0.063
YAL005C,-0.299,0.127,-0.073,-0.227,-0.203,-0.227,-0.076,-0.198,-0.175,0.112,...,-0.069,0.008,0.011,0.020,-0.102,0.026,-0.154,-0.123,0.016,0.212
YAL007C,-0.435,0.146,-0.137,-0.169,-0.194,-0.188,-0.076,-0.191,-0.234,0.117,...,-0.058,-0.139,0.086,-0.187,0.116,0.010,-0.135,0.017,0.053,0.002
YAL008W,-0.126,0.048,-0.007,-0.031,-0.329,-0.025,-0.242,-0.144,-0.155,-0.012,...,0.046,0.048,0.105,-0.080,-0.029,0.016,0.029,-0.104,0.031,0.034


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,499,4776,4775,4771,4767,4765,4777,4759,4758,4752,...,4762,4763,4764,4766,4768,4769,4770,4772,4773,4774
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-1.347,-0.308,0.117,-0.425,-0.327,-1.672,-0.287,-1.268,-0.663,-0.001,...,-0.755,-0.247,-0.380,-0.331,-0.129,-0.118,0.005,-0.035,-0.920,-0.251
1863,YAL004W,-0.167,0.052,-0.102,-0.144,-0.144,-0.055,-0.003,0.110,0.079,0.205,...,0.270,0.420,0.185,-0.058,-0.106,-0.028,0.108,-0.046,0.054,-0.063
4,YAL005C,-0.299,0.127,-0.073,-0.227,-0.203,-0.227,-0.076,-0.198,-0.175,0.112,...,-0.069,0.008,0.011,0.020,-0.102,0.026,-0.154,-0.123,0.016,0.212
5,YAL007C,-0.435,0.146,-0.137,-0.169,-0.194,-0.188,-0.076,-0.191,-0.234,0.117,...,-0.058,-0.139,0.086,-0.187,0.116,0.010,-0.135,0.017,0.053,0.002
6,YAL008W,-0.126,0.048,-0.007,-0.031,-0.329,-0.025,-0.242,-0.144,-0.155,-0.012,...,0.046,0.048,0.105,-0.080,-0.029,0.016,0.029,-0.104,0.031,0.034


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,499,4776,4775,4771,4767,4765,4777,4759,4758,4752,...,4762,4763,4764,4766,4768,4769,4770,4772,4773,4774
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-1.347,-0.308,0.117,-0.425,-0.327,-1.672,-0.287,-1.268,-0.663,-0.001,...,-3.651059,-0.606841,-2.137321,-0.705847,-0.743391,-0.836372,0.152842,-0.370513,-3.200015,-1.585075
1863,YAL004W,-0.167,0.052,-0.102,-0.144,-0.144,-0.055,-0.003,0.110,0.079,0.205,...,1.490582,0.972581,1.086991,-0.000548,-0.620838,-0.164105,0.515544,-0.455049,0.356188,-0.452931
4,YAL005C,-0.299,0.127,-0.073,-0.227,-0.203,-0.227,-0.076,-0.198,-0.175,0.112,...,-0.209922,-0.003014,0.094017,0.200966,-0.599524,0.239254,-0.407057,-1.046799,0.217445,1.203131
5,YAL007C,-0.435,0.146,-0.137,-0.169,-0.194,-0.188,-0.076,-0.191,-0.234,0.117,...,-0.154743,-0.351102,0.522023,-0.333821,0.562064,0.119740,-0.340151,0.029110,0.352537,-0.061498
6,YAL008W,-0.126,0.048,-0.007,-0.031,-0.329,-0.025,-0.242,-0.144,-0.155,-0.012,...,0.366945,0.091704,0.630452,-0.057385,-0.210552,0.164558,0.237355,-0.900782,0.272212,0.131208


# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 16738548...


  0%|          | 0/27 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 27/27 [02:39<00:00,  5.91s/it]

Updating the data_modified_on field...
